In [1]:
from datetime import timedelta
import torch
from torch.autograd import Variable
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, models,transforms
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import TensorDataset, DataLoader,random_split
from __future__ import print_function
import numpy as np 
import pandas as pd
import os
import time
import random
import matplotlib.pyplot as plt 
from torch.autograd import Function
from collections import OrderedDict
import torch.nn as nn
import math
import torchvision.models as models
import pickle
import cv2
import os
import sys
import glob
import collections
import pandas as pd
import wandb
from sklearn.model_selection import KFold
# from torchsummary import summary
import glob
from sklearn.manifold import TSNE
import seaborn as sns
import collections
from transformers import BertConfig, BertModel


import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

from tabulate import tabulate
from tqdm import trange
import random

SAVING_FRAMES_PER_SECOND = 30

In [2]:
VIDEO_FOLDER = "/media/data_cifs/nih/files_to_send/"
ACTION_FOLDER = "/media/data_cifs/nih/inference/results_csv/"

In [3]:
def cohort_search(cohort, search_root, ret=False):
    cohort = str(cohort).upper()
    all_folders = [folder for folder in os.listdir(search_root) if folder.startswith("FC-"+cohort)]
    all_folders.sort()
    all_folder_list = "\n".join(all_folders)
    if ret:
        return all_folders
    else:
        print(all_folder_list)

In [5]:
def find_mouse_videos(mouse_id, cohort, ret=False):
    
    search_for = "v"
    
    if search_for == "videos":
        print("Looking for videos..")
        file_extension = ".mp4" 
        ROOT_FOLDER = VIDEO_FOLDER
    else:
        print("Looking for actions...")
        file_extension = ".csv"
        ROOT_FOLDER = ACTION_FOLDER
    
    mouse_id = str(mouse_id)
    cohort_folders = cohort_search(cohort, ROOT_FOLDER, ret=True)
    
    preexposure_folders = [folder for folder in cohort_folders if "exp" in folder]
    postcond_folders = [folder for folder in cohort_folders if "cond" in folder]
    postext_folders = [folder for folder in cohort_folders if "xt" in folder]
    postret_folders = [folder for folder in cohort_folders if "tre" in folder]
    
    preexposure_videos = []
    postcond_videos = []
    postext_videos = []
    postret_videos = []
    
    for folder in preexposure_folders:
        preexposure_videos += [video+'/'+folder for video in os.listdir(ROOT_FOLDER+folder) if (mouse_id in video and video.endswith(file_extension) and os.stat(ROOT_FOLDER+folder+'/'+video).st_size)]
    preexposure_videos.sort()
    
    for folder in postcond_folders:
        postcond_videos += [video+'/'+folder for video in os.listdir(ROOT_FOLDER+folder) if (mouse_id in video and video.endswith(file_extension) and os.stat(ROOT_FOLDER+folder+'/'+video).st_size)]
    postcond_videos.sort()
    
    for folder in postext_folders:
        postext_videos += [video+'/'+folder for video in os.listdir(ROOT_FOLDER+folder) if (mouse_id in video and video.endswith(file_extension) and os.stat(ROOT_FOLDER+folder+'/'+video).st_size)]
    postext_videos.sort()
    
    for folder in postret_folders:
        postret_videos += [video+'/'+folder for video in os.listdir(ROOT_FOLDER+folder) if (mouse_id in video and video.endswith(file_extension) and os.stat(ROOT_FOLDER+folder+'/'+video).st_size)]
    postret_videos.sort()
    
    if ret:
        return preexposure_videos, postcond_videos
    else:
        print("\nPREEXPOSURE:")
        print("\n".join(preexposure_videos))
        print("\nPOSTCONDITIONING:")
        print("\n".join(postcond_videos))
        print("\nPOSTEXTINCTION:")
        print("\n".join(postext_videos))
        print("\nPOSTRETRIEVAL:")
        print("\n".join(postret_videos))

In [6]:
BEH_LABELS = ['drink', 'eat', 'groom', 'hang', 'sniff', 'rear', 'rest', 'walk', 'nibble']
action_colours = ['tab:blue', 'tab:orange', 'limegreen', 'deepskyblue', 'tab:pink', 'tab:purple', 'tab:brown', 'red', 'black']
len(action_colours) == len(BEH_LABELS)

True

In [ ]:
# READING THE CSV: postcond2_actions[0].iloc[2][1]
# Loading actions
# 17202338:
preexposure_actions_17202338_a = []
postcond_actions_17202338_a = []
postext_actions_17202338_a = []
postret_actions_17202338_a = []

preexposure_video_list_17202338_a = [
"video_2018Y_08M_10D_23h_56m_27s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_00h_56m_27s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_01h_56m_28s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_02h_56m_28s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_03h_56m_28s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_04h_56m_28s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_05h_56m_28s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_06h_56m_29s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_07h_56m_29s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_09h_56m_29s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_10h_56m_29s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_11h_56m_30s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_12h_56m_30s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_13h_56m_30s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_14h_56m_30s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_15h_56m_31s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_16h_56m_31s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_17h_56m_31s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_18h_56m_31s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_19h_56m_31s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_20h_56m_32s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_21h_56m_32s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_22h_56m_32s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_23h_56m_32s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_12D_00h_56m_32s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_12D_01h_56m_33s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_12D_02h_56m_33s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_12D_03h_56m_34s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_12D_04h_56m_34s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_12D_05h_56m_34s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_12D_06h_56m_34s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_12D_07h_56m_35s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
]


for action_list in preexposure_video_list_17202338_a:
    folder_name = action_list.split('/')[-1]
    action_file = action_list.split('/')[0]
    csv_name = ACTION_FOLDER+folder_name+"/"+action_file
    actions = pd.read_csv(csv_name)
    print("Reading CSV:", csv_name)
    preexposure_actions_17202338_a.append(actions)

print("Preexposure Files:", len(preexposure_actions_17202338_a))

postcond_video_list_17202338_a = [
"video_2018Y_08M_13D_23h_21m_05s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_00h_21m_05s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_01h_21m_05s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_02h_21m_05s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_03h_21m_05s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_04h_21m_05s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_05h_21m_05s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_06h_21m_05s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_07h_21m_06s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_09h_28m_54s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_10h_28m_55s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_11h_28m_55s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_12h_28m_56s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_13h_28m_56s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_14h_28m_56s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_15h_28m_57s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_16h_28m_57s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_17h_28m_58s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_18h_28m_58s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_19h_28m_58s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_20h_28m_58s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_21h_28m_58s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_22h_28m_58s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_23h_28m_59s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_15D_00h_28m_59s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_15D_01h_28m_59s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_15D_02h_28m_59s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_15D_03h_28m_59s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_15D_04h_28m_59s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_15D_05h_28m_59s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_15D_06h_29m_00s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_15D_07h_29m_00s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
]

for action_list in postcond_video_list_17202338_a:
    folder_name = action_list.split('/')[-1]
    action_file = action_list.split('/')[0]
    csv_name = ACTION_FOLDER+folder_name+"/"+action_file
    actions = pd.read_csv(csv_name)
    print("Reading CSV:", csv_name)
    postcond_actions_17202338_a.append(actions)
print("Postcond Files:", len(postcond_actions_17202338_a))
    
postext_video_list_17202338_a = [
"video_2018Y_08M_15D_23h_58m_47s_cam_17202338-0000.csv/FC-A_08-15-18_postext",
"video_2018Y_08M_16D_00h_58m_47s_cam_17202338-0000.csv/FC-A_08-15-18_postext",
"video_2018Y_08M_16D_01h_58m_47s_cam_17202338-0000.csv/FC-A_08-15-18_postext",
"video_2018Y_08M_16D_02h_58m_48s_cam_17202338-0000.csv/FC-A_08-15-18_postext",
"video_2018Y_08M_16D_03h_58m_49s_cam_17202338-0000.csv/FC-A_08-15-18_postext",
]

for action_list in postext_video_list_17202338_a:
    folder_name = action_list.split('/')[-1]
    action_file = action_list.split('/')[0]
    csv_name = ACTION_FOLDER+folder_name+"/"+action_file
    actions = pd.read_csv(csv_name)
    print("Reading CSV:", csv_name)
    postext_actions_17202338_a.append(actions)

print("Postext Files:", len(postext_actions_17202338_a))

postret_video_list_17202338_a = [
"video_2018Y_08M_20D_23h_54m_37s_cam_17202338-0000.csv/FC-A_08-20-18_postret",
"video_2018Y_08M_21D_00h_54m_37s_cam_17202338-0000.csv/FC-A_08-20-18_postret",
"video_2018Y_08M_21D_01h_54m_37s_cam_17202338-0000.csv/FC-A_08-20-18_postret",
"video_2018Y_08M_21D_02h_54m_37s_cam_17202338-0000.csv/FC-A_08-20-18_postret",
"video_2018Y_08M_21D_03h_54m_37s_cam_17202338-0000.csv/FC-A_08-20-18_postret",
]

for action_list in postret_video_list_17202338_a:
    folder_name = action_list.split('/')[-1]
    action_file = action_list.split('/')[0]
    csv_name = ACTION_FOLDER+folder_name+"/"+action_file
    actions = pd.read_csv(csv_name)
    print("Reading CSV:", csv_name)
    postret_actions_17202338_a.append(actions)

print("Postret Files:", len(postret_actions_17202338_a))

In [ ]:
# READING THE CSV: postcond2_actions[0].iloc[2][1]
# Loading actions
# 17202338:
preexposure_actions_17202338_b = []
postcond_actions_17202338_b = []
postext_actions_17202338_b = []
postret_actions_17202338_b = []

preexposure_video_list_17202338_b = [
"video_2018Y_09M_19D_23h_02m_56s_cam_17202338-0000.csv/FC-B_1-6_Preexposure_09-17_09-21_2",
"video_2018Y_09M_20D_00h_02m_57s_cam_17202338-0000.csv/FC-B_1-6_Preexposure_09-17_09-21_2",
"video_2018Y_09M_20D_01h_02m_58s_cam_17202338-0000.csv/FC-B_1-6_Preexposure_09-17_09-21_2",
"video_2018Y_09M_20D_02h_03m_00s_cam_17202338-0000.csv/FC-B_1-6_Preexposure_09-17_09-21_2",
"video_2018Y_09M_20D_03h_03m_01s_cam_17202338-0000.csv/FC-B_1-6_Preexposure_09-17_09-21_2",
"video_2018Y_09M_20D_04h_03m_02s_cam_17202338-0000.csv/FC-B_1-6_Preexposure_09-17_09-21_2",
"video_2018Y_09M_20D_05h_03m_03s_cam_17202338-0000.csv/FC-B_1-6_Preexposure_09-17_09-21_2",
"video_2018Y_09M_20D_06h_03m_04s_cam_17202338-0000.csv/FC-B_1-6_Preexposure_09-17_09-21_2",
"video_2018Y_09M_20D_07h_03m_05s_cam_17202338-0000.csv/FC-B_1-6_Preexposure_09-17_09-21_2",
"video_2018Y_09M_20D_08h_03m_05s_cam_17202338-0000.csv/FC-B_1-6_Preexposure_09-17_09-21_2",
"video_2018Y_09M_20D_09h_03m_06s_cam_17202338-0000.csv/FC-B_1-6_Preexposure_09-17_09-21_2",
"video_2018Y_09M_20D_10h_06m_00s_cam_17202338-0000.csv/FC-B_1-6_Preexposure_09-17_09-21_2",
"video_2018Y_09M_20D_11h_06m_03s_cam_17202338-0000.csv/FC-B_1-6_Preexposure_09-17_09-21_2",
"video_2018Y_09M_20D_12h_06m_05s_cam_17202338-0000.csv/FC-B_1-6_Preexposure_09-17_09-21_2",
"video_2018Y_09M_20D_13h_06m_06s_cam_17202338-0000.csv/FC-B_1-6_Preexposure_09-17_09-21_2",
"video_2018Y_09M_20D_14h_06m_08s_cam_17202338-0000.csv/FC-B_1-6_Preexposure_09-17_09-21_2",
"video_2018Y_09M_20D_15h_06m_09s_cam_17202338-0000.csv/FC-B_1-6_Preexposure_09-17_09-21_2",
"video_2018Y_09M_20D_16h_06m_09s_cam_17202338-0000.csv/FC-B_1-6_Preexposure_09-17_09-21_2",
"video_2018Y_09M_20D_17h_06m_10s_cam_17202338-0000.csv/FC-B_1-6_Preexposure_09-17_09-21_2",
"video_2018Y_09M_20D_18h_06m_11s_cam_17202338-0000.csv/FC-B_1-6_Preexposure_09-17_09-21_2",
"video_2018Y_09M_20D_19h_06m_12s_cam_17202338-0000.csv/FC-B_1-6_Preexposure_09-17_09-21_2",
"video_2018Y_09M_20D_20h_06m_13s_cam_17202338-0000.csv/FC-B_1-6_Preexposure_09-17_09-21_2",
"video_2018Y_09M_20D_21h_06m_13s_cam_17202338-0000.csv/FC-B_1-6_Preexposure_09-17_09-21_2",
"video_2018Y_09M_20D_22h_06m_14s_cam_17202338-0000.csv/FC-B_1-6_Preexposure_09-17_09-21_2",
"video_2018Y_09M_20D_23h_06m_15s_cam_17202338-0000.csv/FC-B_1-6_Preexposure_09-17_09-21_2",
"video_2018Y_09M_21D_00h_06m_16s_cam_17202338-0000.csv/FC-B_1-6_Preexposure_09-17_09-21_3",
"video_2018Y_09M_21D_01h_06m_17s_cam_17202338-0000.csv/FC-B_1-6_Preexposure_09-17_09-21_3",
"video_2018Y_09M_21D_02h_06m_18s_cam_17202338-0000.csv/FC-B_1-6_Preexposure_09-17_09-21_3",
"video_2018Y_09M_21D_03h_06m_18s_cam_17202338-0000.csv/FC-B_1-6_Preexposure_09-17_09-21_3",
"video_2018Y_09M_21D_04h_06m_19s_cam_17202338-0000.csv/FC-B_1-6_Preexposure_09-17_09-21_3",
"video_2018Y_09M_21D_05h_06m_20s_cam_17202338-0000.csv/FC-B_1-6_Preexposure_09-17_09-21_3",
"video_2018Y_09M_21D_06h_06m_21s_cam_17202338-0000.csv/FC-B_1-6_Preexposure_09-17_09-21_3",
"video_2018Y_09M_21D_07h_06m_21s_cam_17202338-0000.csv/FC-B_1-6_Preexposure_09-17_09-21_3",
]


for action_list in preexposure_video_list_17202338_b:
    folder_name = action_list.split('/')[-1]
    action_file = action_list.split('/')[0]
    csv_name = ACTION_FOLDER+folder_name+"/"+action_file
    actions = pd.read_csv(csv_name)
    print("Reading CSV:", csv_name)
    preexposure_actions_17202338_b.append(actions)

print("Preexposure Files:", len(preexposure_actions_17202338_b))

postcond_video_list_17202338_b = [
"video_2018Y_09M_21D_23h_39m_45s_cam_17202338-0000.csv/FC-B_1-6_postcond_09-21_09-23_2",
"video_2018Y_09M_22D_00h_39m_45s_cam_17202338-0000.csv/FC-B_1-6_postcond_09-21_09-23_2",
"video_2018Y_09M_22D_01h_39m_46s_cam_17202338-0000.csv/FC-B_1-6_postcond_09-21_09-23_2",
"video_2018Y_09M_22D_02h_39m_47s_cam_17202338-0000.csv/FC-B_1-6_postcond_09-21_09-23_2",
"video_2018Y_09M_22D_03h_39m_47s_cam_17202338-0000.csv/FC-B_1-6_postcond_09-21_09-23_2",
"video_2018Y_09M_22D_04h_39m_48s_cam_17202338-0000.csv/FC-B_1-6_postcond_09-21_09-23_2",
"video_2018Y_09M_22D_05h_39m_49s_cam_17202338-0000.csv/FC-B_1-6_postcond_09-21_09-23_2",
"video_2018Y_09M_22D_06h_39m_49s_cam_17202338-0000.csv/FC-B_1-6_postcond_09-21_09-23_2",
"video_2018Y_09M_22D_07h_39m_50s_cam_17202338-0000.csv/FC-B_1-6_postcond_09-21_09-23_2",
"video_2018Y_09M_22D_08h_39m_51s_cam_17202338-0000.csv/FC-B_1-6_postcond_09-21_09-23_2",
"video_2018Y_09M_22D_09h_39m_51s_cam_17202338-0000.csv/FC-B_1-6_postcond_09-21_09-23_2",
"video_2018Y_09M_22D_10h_39m_52s_cam_17202338-0000.csv/FC-B_1-6_postcond_09-21_09-23_2",
"video_2018Y_09M_22D_11h_39m_53s_cam_17202338-0000.csv/FC-B_1-6_postcond_09-21_09-23_2",
"video_2018Y_09M_22D_12h_39m_54s_cam_17202338-0000.csv/FC-B_1-6_postcond_09-21_09-23_2",
"video_2018Y_09M_22D_13h_39m_54s_cam_17202338-0000.csv/FC-B_1-6_postcond_09-21_09-23_2",
"video_2018Y_09M_22D_14h_39m_55s_cam_17202338-0000.csv/FC-B_1-6_postcond_09-21_09-23_2",
"video_2018Y_09M_22D_15h_39m_55s_cam_17202338-0000.csv/FC-B_1-6_postcond_09-21_09-23_2",
"video_2018Y_09M_22D_16h_39m_56s_cam_17202338-0000.csv/FC-B_1-6_postcond_09-21_09-23_2",
"video_2018Y_09M_22D_17h_39m_57s_cam_17202338-0000.csv/FC-B_1-6_postcond_09-21_09-23_2",
"video_2018Y_09M_22D_18h_39m_57s_cam_17202338-0000.csv/FC-B_1-6_postcond_09-21_09-23_2",
"video_2018Y_09M_22D_19h_39m_58s_cam_17202338-0000.csv/FC-B_1-6_postcond_09-21_09-23_2",
"video_2018Y_09M_22D_20h_39m_59s_cam_17202338-0000.csv/FC-B_1-6_postcond_09-21_09-23_2",
"video_2018Y_09M_22D_21h_39m_59s_cam_17202338-0000.csv/FC-B_1-6_postcond_09-21_09-23_2",
"video_2018Y_09M_22D_22h_40m_00s_cam_17202338-0000.csv/FC-B_1-6_postcond_09-21_09-23_2",
"video_2018Y_09M_22D_23h_40m_00s_cam_17202338-0000.csv/FC-B_1-6_postcond_09-21_09-23_2",
"video_2018Y_09M_23D_00h_40m_01s_cam_17202338-0000.csv/FC-B_1-6_postcond_09-21_09-23_2",
"video_2018Y_09M_23D_01h_40m_02s_cam_17202338-0000.csv/FC-B_1-6_postcond_09-21_09-23_2",
"video_2018Y_09M_23D_02h_40m_02s_cam_17202338-0000.csv/FC-B_1-6_postcond_09-21_09-23_2",
"video_2018Y_09M_23D_03h_40m_03s_cam_17202338-0000.csv/FC-B_1-6_postcond_09-21_09-23_2",
"video_2018Y_09M_23D_04h_40m_04s_cam_17202338-0000.csv/FC-B_1-6_postcond_09-21_09-23_2",
"video_2018Y_09M_23D_05h_40m_04s_cam_17202338-0000.csv/FC-B_1-6_postcond_09-21_09-23_2",
"video_2018Y_09M_23D_06h_40m_05s_cam_17202338-0000.csv/FC-B_1-6_postcond_09-21_09-23_2",
"video_2018Y_09M_23D_07h_40m_05s_cam_17202338-0000.csv/FC-B_1-6_postcond_09-21_09-23_2",
]

for action_list in postcond_video_list_17202338_b:
    folder_name = action_list.split('/')[-1]
    action_file = action_list.split('/')[0]
    csv_name = ACTION_FOLDER+folder_name+"/"+action_file
    actions = pd.read_csv(csv_name)
    print("Reading CSV:", csv_name)
    postcond_actions_17202338_b.append(actions)
print("Postcond Files:", len(postcond_actions_17202338_b))
    
# postext_video_list_17202338_a = [
# "video_2018Y_08M_15D_23h_58m_47s_cam_17202338-0000.csv/FC-A_08-15-18_postext",
# "video_2018Y_08M_16D_00h_58m_47s_cam_17202338-0000.csv/FC-A_08-15-18_postext",
# "video_2018Y_08M_16D_01h_58m_47s_cam_17202338-0000.csv/FC-A_08-15-18_postext",
# "video_2018Y_08M_16D_02h_58m_48s_cam_17202338-0000.csv/FC-A_08-15-18_postext",
# "video_2018Y_08M_16D_03h_58m_49s_cam_17202338-0000.csv/FC-A_08-15-18_postext",
# ]

# for action_list in postext_video_list_17202338_a:
#     folder_name = action_list.split('/')[-1]
#     action_file = action_list.split('/')[0]
#     csv_name = ACTION_FOLDER+folder_name+"/"+action_file
#     actions = pd.read_csv(csv_name)
#     print("Reading CSV:", csv_name)
#     postext_actions_17202338_a.append(actions)

# print("Postext Files:", len(postext_actions_17202338_a))

# postret_video_list_17202338_a = [
# "video_2018Y_08M_20D_23h_54m_37s_cam_17202338-0000.csv/FC-A_08-20-18_postret",
# "video_2018Y_08M_21D_00h_54m_37s_cam_17202338-0000.csv/FC-A_08-20-18_postret",
# "video_2018Y_08M_21D_01h_54m_37s_cam_17202338-0000.csv/FC-A_08-20-18_postret",
# "video_2018Y_08M_21D_02h_54m_37s_cam_17202338-0000.csv/FC-A_08-20-18_postret",
# "video_2018Y_08M_21D_03h_54m_37s_cam_17202338-0000.csv/FC-A_08-20-18_postret",
# ]

# for action_list in postret_video_list_17202338_a:
#     folder_name = action_list.split('/')[-1]
#     action_file = action_list.split('/')[0]
#     csv_name = ACTION_FOLDER+folder_name+"/"+action_file
#     actions = pd.read_csv(csv_name)
#     print("Reading CSV:", csv_name)
#     postret_actions_17202338_a.append(actions)

# print("Postret Files:", len(postret_actions_17202338_a))

In [ ]:
# READING THE CSV: postcond2_actions[0].iloc[2][1]
# Loading actions
# 17202338:
preexposure_actions_17202338_c = []
postcond_actions_17202338_c = []
postext_actions_17202338_c = []
postret_actions_17202338_c = []

preexposure_video_list_17202338_c = [
"video_2018Y_10M_13D_23h_04m_00s_cam_17202338-0000.csv/FC-C_1-6_Preexposure_10-11_10-15",
"video_2018Y_10M_14D_00h_04m_01s_cam_17202338-0000.csv/FC-C_1-6_Preexposure_10-11_10-15",
"video_2018Y_10M_14D_01h_04m_02s_cam_17202338-0000.csv/FC-C_1-6_Preexposure_10-11_10-15",
"video_2018Y_10M_14D_02h_04m_03s_cam_17202338-0000.csv/FC-C_1-6_Preexposure_10-11_10-15",
"video_2018Y_10M_14D_03h_04m_04s_cam_17202338-0000.csv/FC-C_1-6_Preexposure_10-11_10-15",
"video_2018Y_10M_14D_04h_04m_05s_cam_17202338-0000.csv/FC-C_1-6_Preexposure_10-11_10-15",
"video_2018Y_10M_14D_05h_04m_07s_cam_17202338-0000.csv/FC-C_1-6_Preexposure_10-11_10-15",
"video_2018Y_10M_14D_06h_04m_08s_cam_17202338-0000.csv/FC-C_1-6_Preexposure_10-11_10-15",
"video_2018Y_10M_14D_07h_04m_09s_cam_17202338-0000.csv/FC-C_1-6_Preexposure_10-11_10-15",
"video_2018Y_10M_14D_08h_04m_10s_cam_17202338-0000.csv/FC-C_1-6_Preexposure_10-11_10-15",
"video_2018Y_10M_14D_09h_04m_10s_cam_17202338-0000.csv/FC-C_1-6_Preexposure_10-11_10-15",
"video_2018Y_10M_14D_10h_04m_11s_cam_17202338-0000.csv/FC-C_1-6_Preexposure_10-11_10-15",
"video_2018Y_10M_14D_11h_04m_12s_cam_17202338-0000.csv/FC-C_1-6_Preexposure_10-11_10-15",
"video_2018Y_10M_14D_12h_04m_12s_cam_17202338-0000.csv/FC-C_1-6_Preexposure_10-11_10-15",
"video_2018Y_10M_14D_13h_04m_13s_cam_17202338-0000.csv/FC-C_1-6_Preexposure_10-11_10-15",
"video_2018Y_10M_14D_14h_04m_14s_cam_17202338-0000.csv/FC-C_1-6_Preexposure_10-11_10-15",
"video_2018Y_10M_14D_15h_04m_15s_cam_17202338-0000.csv/FC-C_1-6_Preexposure_10-11_10-15",
"video_2018Y_10M_14D_16h_04m_16s_cam_17202338-0000.csv/FC-C_1-6_Preexposure_10-11_10-15",
"video_2018Y_10M_14D_17h_04m_17s_cam_17202338-0000.csv/FC-C_1-6_Preexposure_10-11_10-15",
"video_2018Y_10M_14D_18h_04m_18s_cam_17202338-0000.csv/FC-C_1-6_Preexposure_10-11_10-15",
"video_2018Y_10M_14D_19h_04m_18s_cam_17202338-0000.csv/FC-C_1-6_Preexposure_10-11_10-15",
"video_2018Y_10M_14D_20h_04m_19s_cam_17202338-0000.csv/FC-C_1-6_Preexposure_10-11_10-15",
"video_2018Y_10M_14D_21h_04m_20s_cam_17202338-0000.csv/FC-C_1-6_Preexposure_10-11_10-15",
"video_2018Y_10M_14D_22h_04m_21s_cam_17202338-0000.csv/FC-C_1-6_Preexposure_10-11_10-15",
"video_2018Y_10M_14D_23h_04m_22s_cam_17202338-0000.csv/FC-C_1-6_Preexposure_10-11_10-15",
"video_2018Y_10M_15D_00h_04m_22s_cam_17202338-0000.csv/FC-C_1-6_Preexposure_10-11_10-15",
"video_2018Y_10M_15D_01h_04m_23s_cam_17202338-0000.csv/FC-C_1-6_Preexposure_10-11_10-15",
"video_2018Y_10M_15D_02h_04m_24s_cam_17202338-0000.csv/FC-C_1-6_Preexposure_10-11_10-15",
"video_2018Y_10M_15D_03h_04m_25s_cam_17202338-0000.csv/FC-C_1-6_Preexposure_10-11_10-15",
"video_2018Y_10M_15D_04h_04m_25s_cam_17202338-0000.csv/FC-C_1-6_Preexposure_10-11_10-15",
"video_2018Y_10M_15D_05h_04m_26s_cam_17202338-0000.csv/FC-C_1-6_Preexposure_10-11_10-15",
"video_2018Y_10M_15D_06h_04m_27s_cam_17202338-0000.csv/FC-C_1-6_Preexposure_10-11_10-15",
]


for action_list in preexposure_video_list_17202338_c:
    folder_name = action_list.split('/')[-1]
    action_file = action_list.split('/')[0]
    csv_name = ACTION_FOLDER+folder_name+"/"+action_file
    actions = pd.read_csv(csv_name)
    print("Reading CSV:", csv_name)
    preexposure_actions_17202338_c.append(actions)

print("Preexposure Files:", len(preexposure_actions_17202338_c))

postcond_video_list_17202338_c = [
"video_2018Y_10M_15D_23h_16m_17s_cam_17202338-0000.csv/FC-C_1-6_postcond_10-15_10-17",
"video_2018Y_10M_16D_00h_16m_18s_cam_17202338-0000.csv/FC-C_1-6_postcond_10-15_10-17",
"video_2018Y_10M_16D_01h_16m_18s_cam_17202338-0000.csv/FC-C_1-6_postcond_10-15_10-17",
"video_2018Y_10M_16D_02h_16m_19s_cam_17202338-0000.csv/FC-C_1-6_postcond_10-15_10-17",
"video_2018Y_10M_16D_03h_16m_20s_cam_17202338-0000.csv/FC-C_1-6_postcond_10-15_10-17",
"video_2018Y_10M_16D_04h_16m_21s_cam_17202338-0000.csv/FC-C_1-6_postcond_10-15_10-17",
"video_2018Y_10M_16D_05h_16m_21s_cam_17202338-0000.csv/FC-C_1-6_postcond_10-15_10-17",
"video_2018Y_10M_16D_06h_16m_22s_cam_17202338-0000.csv/FC-C_1-6_postcond_10-15_10-17",
"video_2018Y_10M_16D_07h_16m_23s_cam_17202338-0000.csv/FC-C_1-6_postcond_10-15_10-17",
"video_2018Y_10M_16D_08h_16m_24s_cam_17202338-0000.csv/FC-C_1-6_postcond_10-15_10-17",
"video_2018Y_10M_16D_09h_16m_25s_cam_17202338-0000.csv/FC-C_1-6_postcond_10-15_10-17",
"video_2018Y_10M_16D_10h_35m_40s_cam_17202338-0000.csv/FC-C_1-6_postcond_10-15_10-17",
"video_2018Y_10M_16D_11h_35m_43s_cam_17202338-0000.csv/FC-C_1-6_postcond_10-15_10-17",
"video_2018Y_10M_16D_12h_35m_45s_cam_17202338-0000.csv/FC-C_1-6_postcond_10-15_10-17",
"video_2018Y_10M_16D_13h_35m_48s_cam_17202338-0000.csv/FC-C_1-6_postcond_10-15_10-17",
"video_2018Y_10M_16D_14h_35m_49s_cam_17202338-0000.csv/FC-C_1-6_postcond_10-15_10-17",
"video_2018Y_10M_16D_15h_35m_50s_cam_17202338-0000.csv/FC-C_1-6_postcond_10-15_10-17",
"video_2018Y_10M_16D_16h_35m_50s_cam_17202338-0000.csv/FC-C_1-6_postcond_10-15_10-17",
"video_2018Y_10M_16D_17h_35m_51s_cam_17202338-0000.csv/FC-C_1-6_postcond_10-15_10-17",
"video_2018Y_10M_16D_18h_35m_52s_cam_17202338-0000.csv/FC-C_1-6_postcond_10-15_10-17",
"video_2018Y_10M_16D_19h_35m_53s_cam_17202338-0000.csv/FC-C_1-6_postcond_10-15_10-17",
"video_2018Y_10M_16D_20h_35m_54s_cam_17202338-0000.csv/FC-C_1-6_postcond_10-15_10-17",
"video_2018Y_10M_16D_21h_35m_55s_cam_17202338-0000.csv/FC-C_1-6_postcond_10-15_10-17",
"video_2018Y_10M_16D_22h_35m_56s_cam_17202338-0000.csv/FC-C_1-6_postcond_10-15_10-17",
"video_2018Y_10M_16D_23h_35m_56s_cam_17202338-0000.csv/FC-C_1-6_postcond_10-15_10-17",
"video_2018Y_10M_17D_00h_35m_57s_cam_17202338-0000.csv/FC-C_1-6_postcond_10-15_10-17",
"video_2018Y_10M_17D_01h_35m_58s_cam_17202338-0000.csv/FC-C_1-6_postcond_10-15_10-17",
"video_2018Y_10M_17D_02h_35m_58s_cam_17202338-0000.csv/FC-C_1-6_postcond_10-15_10-17",
"video_2018Y_10M_17D_03h_35m_59s_cam_17202338-0000.csv/FC-C_1-6_postcond_10-15_10-17",
"video_2018Y_10M_17D_04h_35m_59s_cam_17202338-0000.csv/FC-C_1-6_postcond_10-15_10-17",
"video_2018Y_10M_17D_05h_36m_00s_cam_17202338-0000.csv/FC-C_1-6_postcond_10-15_10-17",
"video_2018Y_10M_17D_06h_36m_01s_cam_17202338-0000.csv/FC-C_1-6_postcond_10-15_10-17",
]

for action_list in postcond_video_list_17202338_c:
    folder_name = action_list.split('/')[-1]
    action_file = action_list.split('/')[0]
    csv_name = ACTION_FOLDER+folder_name+"/"+action_file
    actions = pd.read_csv(csv_name)
    print("Reading CSV:", csv_name)
    postcond_actions_17202338_c.append(actions)
print("Postcond Files:", len(postcond_actions_17202338_c))
    
# postext_video_list_17202338_a = [
# "video_2018Y_08M_15D_23h_58m_47s_cam_17202338-0000.csv/FC-A_08-15-18_postext",
# "video_2018Y_08M_16D_00h_58m_47s_cam_17202338-0000.csv/FC-A_08-15-18_postext",
# "video_2018Y_08M_16D_01h_58m_47s_cam_17202338-0000.csv/FC-A_08-15-18_postext",
# "video_2018Y_08M_16D_02h_58m_48s_cam_17202338-0000.csv/FC-A_08-15-18_postext",
# "video_2018Y_08M_16D_03h_58m_49s_cam_17202338-0000.csv/FC-A_08-15-18_postext",
# ]

# for action_list in postext_video_list_17202338_a:
#     folder_name = action_list.split('/')[-1]
#     action_file = action_list.split('/')[0]
#     csv_name = ACTION_FOLDER+folder_name+"/"+action_file
#     actions = pd.read_csv(csv_name)
#     print("Reading CSV:", csv_name)
#     postext_actions_17202338_a.append(actions)

# print("Postext Files:", len(postext_actions_17202338_a))

# postret_video_list_17202338_a = [
# "video_2018Y_08M_20D_23h_54m_37s_cam_17202338-0000.csv/FC-A_08-20-18_postret",
# "video_2018Y_08M_21D_00h_54m_37s_cam_17202338-0000.csv/FC-A_08-20-18_postret",
# "video_2018Y_08M_21D_01h_54m_37s_cam_17202338-0000.csv/FC-A_08-20-18_postret",
# "video_2018Y_08M_21D_02h_54m_37s_cam_17202338-0000.csv/FC-A_08-20-18_postret",
# "video_2018Y_08M_21D_03h_54m_37s_cam_17202338-0000.csv/FC-A_08-20-18_postret",
# ]

# for action_list in postret_video_list_17202338_a:
#     folder_name = action_list.split('/')[-1]
#     action_file = action_list.split('/')[0]
#     csv_name = ACTION_FOLDER+folder_name+"/"+action_file
#     actions = pd.read_csv(csv_name)
#     print("Reading CSV:", csv_name)
#     postret_actions_17202338_a.append(actions)

# print("Postret Files:", len(postret_actions_17202338_a))

In [ ]:
# Loading actions CONTROL
# 17202338:
preexposure_actions_17202338_k = []
postcond_actions_17202338_k = []
postext_actions_17202338_k = []
postret_actions_17202338_k = []

preexposure_video_list_17202338_k = [
"video_2019Y_06M_04D_20h_14m_53s_cam_6394840-0000.csv/FC-D_preexposure_W1_06-04-2019_comp2",
"video_2019Y_06M_04D_21h_14m_55s_cam_6394840-0000.csv/FC-D_preexposure_W1_06-04-2019_comp2",
"video_2019Y_06M_04D_22h_14m_56s_cam_6394840-0000.csv/FC-D_preexposure_W1_06-04-2019_comp2",
"video_2019Y_06M_04D_23h_15m_00s_cam_6394840-0000.csv/FC-D_preexposure_W1_06-04-2019_comp2",
"video_2019Y_06M_05D_00h_15m_01s_cam_6394840-0000.csv/FC-D_preexposure_W1_06-04-2019_comp2",
"video_2019Y_06M_05D_01h_15m_02s_cam_6394840-0000.csv/FC-D_preexposure_W1_06-04-2019_comp2",
"video_2019Y_06M_05D_02h_15m_03s_cam_6394840-0000.csv/FC-D_preexposure_W1_06-04-2019_comp2",
"video_2019Y_06M_05D_03h_15m_08s_cam_6394840-0000.csv/FC-D_preexposure_W1_06-04-2019_comp2",
"video_2019Y_06M_05D_04h_15m_09s_cam_6394840-0000.csv/FC-D_preexposure_W1_06-04-2019_comp2",
"video_2019Y_06M_05D_05h_15m_14s_cam_6394840-0000.csv/FC-D_preexposure_W1_06-04-2019_comp2",
"video_2019Y_06M_05D_06h_15m_17s_cam_6394840-0000.csv/FC-D_preexposure_W1_06-04-2019_comp2",
"video_2019Y_06M_05D_07h_15m_18s_cam_6394840-0000.csv/FC-D_preexposure_W1_06-04-2019_comp2",
"video_2019Y_06M_05D_08h_15m_19s_cam_6394840-0000.csv/FC-D_preexposure_W1_06-04-2019_comp2",
"video_2019Y_06M_05D_09h_15m_20s_cam_6394840-0000.csv/FC-D_preexposure_W1_06-04-2019_comp2",
"video_2019Y_06M_05D_10h_15m_21s_cam_6394840-0000.csv/FC-D_preexposure_W1_06-04-2019_comp2",
"video_2019Y_06M_05D_11h_15m_22s_cam_6394840-0000.csv/FC-D_preexposure_W1_06-04-2019_comp2",
"video_2019Y_06M_05D_13h_23m_04s_cam_6394840-0000.csv/FC-D_preexposure_W1_06-05-2019_comp2",
"video_2019Y_06M_05D_14h_23m_05s_cam_6394840-0000.csv/FC-D_preexposure_W1_06-05-2019_comp2",
"video_2019Y_06M_05D_15h_23m_08s_cam_6394840-0000.csv/FC-D_preexposure_W1_06-05-2019_comp2",
"video_2019Y_06M_05D_16h_23m_12s_cam_6394840-0000.csv/FC-D_preexposure_W1_06-05-2019_comp2",
"video_2019Y_06M_05D_17h_23m_13s_cam_6394840-0000.csv/FC-D_preexposure_W1_06-05-2019_comp2",
"video_2019Y_06M_05D_18h_23m_17s_cam_6394840-0000.csv/FC-D_preexposure_W1_06-05-2019_comp2",
"video_2019Y_06M_05D_19h_23m_20s_cam_6394840-0000.csv/FC-D_preexposure_W1_06-05-2019_comp2",
"video_2019Y_06M_05D_20h_23m_22s_cam_6394840-0000.csv/FC-D_preexposure_W1_06-05-2019_comp2",
"video_2019Y_06M_05D_21h_23m_24s_cam_6394840-0000.csv/FC-D_preexposure_W1_06-05-2019_comp2",
"video_2019Y_06M_05D_22h_23m_25s_cam_6394840-0000.csv/FC-D_preexposure_W1_06-05-2019_comp2",
"video_2019Y_06M_05D_23h_23m_26s_cam_6394840-0000.csv/FC-D_preexposure_W1_06-05-2019_comp2",
"video_2019Y_06M_06D_00h_23m_28s_cam_6394840-0000.csv/FC-D_preexposure_W1_06-05-2019_comp2",
"video_2019Y_06M_06D_01h_23m_31s_cam_6394840-0000.csv/FC-D_preexposure_W1_06-05-2019_comp2",
"video_2019Y_06M_06D_02h_23m_35s_cam_6394840-0000.csv/FC-D_preexposure_W1_06-05-2019_comp2",
"video_2019Y_06M_06D_03h_23m_38s_cam_6394840-0000.csv/FC-D_preexposure_W1_06-05-2019_comp2",
"video_2019Y_06M_06D_04h_23m_42s_cam_6394840-0000.csv/FC-D_preexposure_W1_06-05-2019_comp2",
]


for action_list in preexposure_video_list_17202338_k:
    folder_name = action_list.split('/')[-1]
    action_file = action_list.split('/')[0]
    csv_name = ACTION_FOLDER+folder_name+"/"+action_file
    actions = pd.read_csv(csv_name)
    print("Reading CSV:", csv_name)
    preexposure_actions_17202338_k.append(actions)

print("Preexposure Files:", len(preexposure_actions_17202338_k))

postcond_video_list_17202338_k = [
"video_2019Y_06M_06D_20h_14m_02s_cam_6394840-0000.csv/FC-D_postcond_W2_06-06-2019_comp2",
"video_2019Y_06M_06D_21h_14m_03s_cam_6394840-0000.csv/FC-D_postcond_W2_06-06-2019_comp2",
"video_2019Y_06M_06D_22h_14m_05s_cam_6394840-0000.csv/FC-D_postcond_W2_06-06-2019_comp2",
"video_2019Y_06M_06D_23h_14m_08s_cam_6394840-0000.csv/FC-D_postcond_W2_06-06-2019_comp2",
"video_2019Y_06M_07D_00h_14m_11s_cam_6394840-0000.csv/FC-D_postcond_W2_06-06-2019_comp2",
"video_2019Y_06M_07D_01h_14m_14s_cam_6394840-0000.csv/FC-D_postcond_W2_06-06-2019_comp2",
"video_2019Y_06M_07D_02h_14m_14s_cam_6394840-0000.csv/FC-D_postcond_W2_06-06-2019_comp2",
"video_2019Y_06M_07D_03h_14m_15s_cam_6394840-0000.csv/FC-D_postcond_W2_06-06-2019_comp2",
"video_2019Y_06M_07D_04h_14m_17s_cam_6394840-0000.csv/FC-D_postcond_W2_06-06-2019_comp2",
"video_2019Y_06M_07D_05h_14m_18s_cam_6394840-0000.csv/FC-D_postcond_W2_06-06-2019_comp2",
"video_2019Y_06M_07D_06h_14m_21s_cam_6394840-0000.csv/FC-D_postcond_W2_06-06-2019_comp2",
"video_2019Y_06M_07D_07h_14m_21s_cam_6394840-0000.csv/FC-D_postcond_W2_06-06-2019_comp2",
"video_2019Y_06M_07D_08h_14m_24s_cam_6394840-0000.csv/FC-D_postcond_W2_06-06-2019_comp2",
"video_2019Y_06M_07D_09h_14m_26s_cam_6394840-0000.csv/FC-D_postcond_W2_06-06-2019_comp2",
"video_2019Y_06M_07D_10h_14m_27s_cam_6394840-0000.csv/FC-D_postcond_W2_06-06-2019_comp2",
"video_2019Y_06M_07D_11h_14m_27s_cam_6394840-0000.csv/FC-D_postcond_W2_06-06-2019_comp2",
"video_2019Y_06M_07D_13h_23m_06s_cam_6394840-0000.csv/FC-D_postcond_W2_06-07-2019_comp2",
"video_2019Y_06M_07D_14h_23m_10s_cam_6394840-0000.csv/FC-D_postcond_W2_06-07-2019_comp2",
"video_2019Y_06M_07D_15h_23m_13s_cam_6394840-0000.csv/FC-D_postcond_W2_06-07-2019_comp2",
"video_2019Y_06M_07D_16h_23m_15s_cam_6394840-0000.csv/FC-D_postcond_W2_06-07-2019_comp2",
"video_2019Y_06M_07D_17h_23m_18s_cam_6394840-0000.csv/FC-D_postcond_W2_06-07-2019_comp2",
"video_2019Y_06M_07D_18h_23m_23s_cam_6394840-0000.csv/FC-D_postcond_W2_06-07-2019_comp2",
"video_2019Y_06M_07D_19h_23m_24s_cam_6394840-0000.csv/FC-D_postcond_W2_06-07-2019_comp2",
"video_2019Y_06M_07D_20h_23m_24s_cam_6394840-0000.csv/FC-D_postcond_W2_06-07-2019_comp2",
"video_2019Y_06M_07D_21h_23m_29s_cam_6394840-0000.csv/FC-D_postcond_W2_06-07-2019_comp2",
"video_2019Y_06M_07D_22h_23m_30s_cam_6394840-0000.csv/FC-D_postcond_W2_06-07-2019_comp2",
"video_2019Y_06M_07D_23h_23m_31s_cam_6394840-0000.csv/FC-D_postcond_W2_06-07-2019_comp2",
"video_2019Y_06M_08D_00h_23m_31s_cam_6394840-0000.csv/FC-D_postcond_W2_06-07-2019_comp2",
"video_2019Y_06M_08D_01h_23m_33s_cam_6394840-0000.csv/FC-D_postcond_W2_06-07-2019_comp2",
"video_2019Y_06M_08D_02h_23m_36s_cam_6394840-0000.csv/FC-D_postcond_W2_06-07-2019_comp2",
"video_2019Y_06M_08D_03h_23m_38s_cam_6394840-0000.csv/FC-D_postcond_W2_06-07-2019_comp2",
"video_2019Y_06M_08D_04h_23m_39s_cam_6394840-0000.csv/FC-D_postcond_W2_06-07-2019_comp2",
]

for action_list in postcond_video_list_17202338_k:
    folder_name = action_list.split('/')[-1]
    action_file = action_list.split('/')[0]
    csv_name = ACTION_FOLDER+folder_name+"/"+action_file
    actions = pd.read_csv(csv_name)
    print("Reading CSV:", csv_name)
    postcond_actions_17202338_k.append(actions)
print("Postcond Files:", len(postcond_actions_17202338_k))

In [ ]:
a = pd.read_csv("/cifs/data/tserre/CLPS_Serre_Lab/nih/inference/results_csv/FC-A_08-10-18_postcond/video_2018Y_08M_14D_03h_21m_05s_cam_17202338-0000.csv")


In [ ]:
print("walk walk walk walk walk rear rear rear rear rear rear rear rear walk walk walk walk walk")

In [ ]:
all_preexposure_actions_17202338_a = np.load("all_preexposure_actions_17202338_a.npy")

# for action_set in preexposure_actions_17202338_a:
#     for idx in range(len(action_set)):
#         all_preexposure_actions_17202338_a.append(action_set.iloc[idx][1])

print("Preexposure:", len(all_preexposure_actions_17202338_a))

all_postcond_actions_17202338_a = np.load("all_postcond_actions_17202338_a.npy")
# for action_set in postcond_actions_17202338_a:
#     for idx in range(len(action_set)):
#         all_postcond_actions_17202338_a.append(action_set.iloc[idx][1])

print("Postcond:", len(all_postcond_actions_17202338_a))

# all_postext_actions_17202338_a = []
# for action_set in postext_actions_17202338_a:
#     for idx in range(len(action_set)):
#         all_postext_actions_17202338_a.append(action_set.iloc[idx][1])
# print("Postext:", len(all_postext_actions_17202338_a))

# all_postret_actions_17202338_a = []
# for action_set in postret_actions_17202338_a:
#     for idx in range(len(action_set)):
#         all_postret_actions_17202338_a.append(action_set.iloc[idx][1])
# print("Postret:", len(all_postret_actions_17202338_a))

In [ ]:
all_preexposure_actions_17202338_b = np.load("all_preexposure_actions_17202338_b.npy")
all_postcond_actions_17202338_b = np.load("all_postcond_actions_17202338_b.npy")

In [ ]:
all_preexposure_actions_17202338_c = np.load("all_preexposure_actions_17202338_c.npy")
all_postcond_actions_17202338_c = np.load("all_postcond_actions_17202338_c.npy")

In [ ]:
# np.save("all_preexposure_actions_17202338_d.npy", all_preexposure_actions_17202338_k)
# np.save("all_postcond_actions_17202338_d.npy", all_preexposure_actions_17202338_k)

In [ ]:
a

In [ ]:
b = []
for idx in range(len(a)):
    b.append(a.iloc[idx][1])
print("Preexposure:", len(b))

In [ ]:
b[:100]

In [ ]:
all_preexposure_actions_17202338_c = []
all_postcond_actions_17202338_c = []

for action_set in preexposure_actions_17202338_c:
    for idx in range(len(action_set)):
        all_preexposure_actions_17202338_c.append(action_set.iloc[idx][1])
print("Preexposure:", len(all_preexposure_actions_17202338_c))

for act ion_set in postcond_actions_17202338_c:
    for idx in range(len(action_set)):
        all_postcond_actions_17202338_c.append(action_set.iloc[idx][1])
print("Postcond:", len(all_postcond_actions_17202338_c))

In [ ]:
all_preexposure_actions_17202338_b = []
all_postcond_actions_17202338_b = []

for action_set in preexposure_actions_17202338_b:
    for idx in range(len(action_set)):
        all_preexposure_actions_17202338_b.append(action_set.iloc[idx][1])
print("Preexposure:", len(all_preexposure_actions_17202338_b))

for action_set in postcond_actions_17202338_b:
    for idx in range(len(action_set)):
        all_postcond_actions_17202338_b.append(action_set.iloc[idx][1])
print("Postcond:", len(all_postcond_actions_17202338_b))

In [ ]:
all_preexposure_actions_17202338_k = []
all_postcond_actions_17202338_k = []

for action_set in preexposure_actions_17202338_k:
    for idx in range(len(action_set)):
        all_preexposure_actions_17202338_k.append(action_set.iloc[idx][1])
print("Preexposure:", len(all_preexposure_actions_17202338_k))

for action_set in postcond_actions_17202338_k:
    for idx in range(len(action_set)):
        all_postcond_actions_17202338_k.append(action_set.iloc[idx][1])
print("Postcond:", len(all_postcond_actions_17202338_k))

In [ ]:
time_counter = {
    'under5':0,
    '5-10':0,
    '10-15':0,
    '15-20':0,
    '20-25':0,
    '25-30':0,
    '30-35':0,
    '35-40':0,
    '40-45':0,
    '45-50':0,
    '50-55':0,
    '55-60':0,
    '60-90':0,
    '90-450':0,
    '450-1800':0,
    '1800-4500':0,
    'over4500':0,}

def get_repeat_vocab(repeat_number):
    # > '0-5'
    # > '5-15'
    # > '15-45'
    # > '45-90'
    # > '90-450'
    # > '450-1800'
    # > '1800-4500'
    # > 'over4500'
    if repeat_number > 0 and repeat_number <= 5:
        box = "under5"
    elif repeat_number > 5 and repeat_number <= 10:
        box = "5-10"
    elif repeat_number > 10 and repeat_number <= 15:
        box = "10-15"
    elif repeat_number > 15 and repeat_number <= 20:
        box = "15-20"
    elif repeat_number > 20 and repeat_number <= 25:
        box = "20-25"
    elif repeat_number > 25 and repeat_number <= 30:
        box = "25-30"
    elif repeat_number > 30 and repeat_number <= 35:
        box = "30-35"
    elif repeat_number > 35 and repeat_number <= 40:
        box = "35-40"
    elif repeat_number > 40 and repeat_number <= 45:
        box = "40-45"
    elif repeat_number > 45 and repeat_number <= 50:
        box = "45-50"
    elif repeat_number > 50 and repeat_number <= 55:
        box = "50-55"
    elif repeat_number > 55 and repeat_number <= 60:
        box = "55-60"
    elif repeat_number > 60 and repeat_number <= 90:
        box = "60-90"
    elif repeat_number > 90 and repeat_number <= 450:
        box = "90-450"
    elif repeat_number > 450 and repeat_number <= 1800:
        box = "450-1800"
    elif repeat_number > 1800 and repeat_number <= 4500:
        box = "1800-4500"
    elif repeat_number > 4500:
        box = "over4500"
    else:
        raise ValueError('A very specific bad thing happened.')
    time_counter[box] += 1
    return box

In [ ]:
def cleaning_actions(action_list):time_counter = {
    'under5':0,
    '5-10':0,
    '10-15':0,
    '15-20':0,
    '20-25':0,
    '25-30':0,
    '30-35':0,
    '35-40':0,
    '40-45':0,
    '45-50':0,
    '50-55':0,
    '55-60':0,
    '60-90':0,
    '90-450':0,
    '450-1800':0,
    '1800-4500':0,
    'over4500':0,}

def get_repeat_vocab(repeat_number):
    # > '0-5'
    # > '5-15'
    # > '15-45'
    # > '45-90'
    # > '90-450'
    # > '450-1800'
    # > '1800-4500'
    # > 'over4500'
    if repeat_number > 0 and repeat_number <= 5:
        box = "under5"
    elif repeat_number > 5 and repeat_number <= 10:
        box = "5-10"
    elif repeat_number > 10 and repeat_number <= 15:
        box = "10-15"
    elif repeat_number > 15 and repeat_number <= 20:
        box = "15-20"
    elif repeat_number > 20 and repeat_number <= 25:
        box = "20-25"
    elif repeat_number > 25 and repeat_number <= 30:
        box = "25-30"
    elif repeat_number > 30 and repeat_number <= 35:
        box = "30-35"
    elif repeat_number > 35 and repeat_number <= 40:
        box = "35-40"
    elif repeat_number > 40 and repeat_number <= 45:
        box = "40-45"
    elif repeat_number > 45 and repeat_number <= 50:
        box = "45-50"
    elif repeat_number > 50 and repeat_number <= 55:
        box = "50-55"
    elif repeat_number > 55 and repeat_number <= 60:
        box = "55-60"
    elif repeat_number > 60 and repeat_number <= 90:
        box = "60-90"
    elif repeat_number > 90 and repeat_number <= 450:
        box = "90-450"
    elif repeat_number > 450 and repeat_number <= 1800:
        box = "450-1800"
    elif repeat_number > 1800 and repeat_number <= 4500:
        box = "1800-4500"
    elif repeat_number > 4500:
        box = "over4500"
    else:
        raise ValueError('A very specific bad thing happened.')
    time_counter[box] += 1
    return box
    WINDOW_SIZE = 3 # Each side
    
    index = WINDOW_SIZE
    action_length = len(action_list)
    
    while index < (action_length - WINDOW_SIZE - 1):
        current_action = action_list[index]
        window = action_list[index - WINDOW_SIZE : index + WINDOW_SIZE + 1]
        window_elements = collections.Counter(window)
        most_common_action = window_elements.most_common(1)[0][0]
        if action_list[index] != most_common_action:
            action_list[index] = most_common_action
        index += 1
    return action_list

In [ ]:
def generate_language_from_labels(action_list, class_label):
    SENTENCE_LENGTH = 900 # in terms of actions
    
    start_index = 0
    action_length = len(action_list)
    
    language = []
    label = []
    sentence = ""
    
    current_sentence_length = 0
    while start_index < action_length:
        
        current_action = action_list[start_index]
        current_action_label = BEH_LABELS[current_action]
        
        next_action_idx = start_index + 1
        
        while next_action_idx < action_length and current_action == action_list[next_action_idx]:
            next_action_idx  += 1
        repeats = next_action_idx - start_index
        
        # Filtering out single frame noise
#         if repeats == 1 and current_action != action_list[next_action_idx]:
#             start_index = next_action_idx
#             print("Single action:", action_list[next_action_idx-2], current_action, action_list[next_action_idx])
#             continue
        
        if current_sentence_length == 0:
            sentence = ""
            
        current_sentence_length += repeats
        start_index = next_action_idx
        
        sentence += str(current_action_label)+" "+get_repeat_vocab(repeats)+" "
        
        if current_sentence_length >= SENTENCE_LENGTH:
            language.append(sentence)
            label.append(class_label)
            current_sentence_length = 0
            
    if sentence != "":
        language.append(sentence)
        label.append(class_label)
    return language, label

In [ ]:
# print("Language Length:", len(generate_language_from_labels(all_preexposure_actions_17202338_a, 0)))
# print("Original Action Length:", len(all_preexposure_actions_17202338_a))

In [ ]:
all_preexposure_actions_17202338_a = np.load("all_preexposure_actions_17202338_a.npy")
all_postcond_actions_17202338_a = np.load("all_postcond_actions_17202338_a.npy")

In [ ]:
cleaned_preexposure = cleaning_actions(all_preexposure_actions_17202338_a)
print("Preexposure done.")
cleaned_postcond = cleaning_actions(all_postcond_actions_17202338_a)
print("Postcond done.")

In [ ]:
cleaned_preexposure_b = cleaning_actions(all_preexposure_actions_17202338_b)
print("Preexposure done.")
cleaned_postcond_b = cleaning_actions(all_postcond_actions_17202338_b)
print("Postcond done.")

In [ ]:
cleaned_preexposure_c = cleaning_actions(all_preexposure_actions_17202338_c)
print("Preexposure done.")
cleaned_postcond_c = cleaning_actions(all_postcond_actions_17202338_c)
print("Postcond done.")


In [ ]:
cleaned_preexposure_d = cleaning_actions(all_preexposure_actions_17202338_d)
print("Preexposure done.")
cleaned_postcond_d = cleaning_actions(all_postcond_actions_17202338_d)
print("Postcond done.")

In [ ]:
raw_text = []
labels = []

In [ ]:
raw_text_b = []
labels_b = []

In [ ]:
raw_text_c = []
labels_c = []

In [ ]:
raw_text_k = []
labels_k = []

In [ ]:
preexposure = generate_language_from_labels(cleaned_preexposure, 0)
preexp_time_counter = time_counter
raw_text += preexposure[0]
labels += preexposure[1]

In [ ]:
postcond = generate_language_from_labels(cleaned_postcond, 1)
postcond_time_counter = time_counter
raw_text += postcond[0]
labels += postcond[1]

In [ ]:
# FC-B
preexposure_b = generate_language_from_labels(cleaned_preexposure_b, 0)
preexp_time_counter_b = time_counter
raw_text_b += preexposure_b[0]
labels_b += preexposure_b[1]

In [ ]:
postcond_b = generate_language_from_labels(cleaned_postcond_b, 1)
postcond_time_counter_b = time_counter
raw_text_b += postcond_b[0]
labels_b += postcond_b[1]

In [ ]:
# FC-C
preexposure_c = generate_language_from_labels(cleaned_preexposure_c, 0)
preexp_time_counter_c = time_counter
raw_text_c += preexposure_c[0]
labels_c += preexposure_c[1]

In [ ]:
postcond_c = generate_language_from_labels(cleaned_postcond_c, 1)
postcond_time_counter_c = time_counter
raw_text_c += postcond_c[0]
labels_c += postcond_c[1]

In [ ]:
# FC-D (k)
preexposure_k = generate_language_from_labels(cleaned_preexposure_k, 0)
preexp_time_counter_k = time_counter
raw_text_k += preexposure_k[0]
labels_k += preexposure_k[1]

In [ ]:
postcond_k = generate_language_from_labels(cleaned_postcond_k, 1)
postcond_time_counter_k = time_counter
raw_text_k += postcond_k[0]
labels_k += postcond_k[1]

In [ ]:
preexp_time_counter

In [ ]:
postcond_time_counter

In [ ]:
len(raw_text), len(labels), len(raw_text) == len(labels)

In [ ]:
print(time_counter)

In [ ]:
# Seeing max sequence length
max_seq_len = 0
seq_lens = []
overall_set = set()
for seq in preexposure[0]:
    overall_set.update(seq.split())
    seq_lens.append(len(seq))
    if len(seq) > max_seq_len:
        max_seq_len = len(seq)
print("Maximum sequence length:", max_seq_len)
seq_lens = np.array(seq_lens)
print("Average:", np.mean(seq_lens))
print("Standard Deviation:", np.std(seq_lens))
print("Final Vocabulary:", overall_set)
print("Length of Vocabulary:", len(overall_set))

In [ ]:
# Plot time distributions:
times = [time_counter[x] for x in time_counter.keys()]

X = [x for x in time_counter.keys()]
X_axis = np.arange(len(X))

plt.figure(figsize=(30, 15))
plt.rcParams.update({'font.size': 22})
plt.bar(X_axis , times, 0.4, label = 'Preexposure')

plt.xticks(X_axis, X)
plt.xlabel("Time Buckets of Actions in seconds")
plt.ylabel("Frequency")
plt.title("Action Frequency in Different Phases of Experiment Mouse")
plt.legend()
plt.show()

In [ ]:
def plotting_action_frequencies_d(action):
    # Analyzing rests - Preexposure
    preexp_rest_time_counter_k = {
        'under5':0,
        '5-10':0,
        '10-15':0,
        '15-20':0,
        '20-25':0,
        '25-30':0,
        '30-35':0,
        '35-40':0,
        '40-45':0,
        '45-50':0,
        '50-55':0,
        '55-60':0,
        '60-90':0,
        '90-450':0,
        '450-1800':0,
        '1800-4500':0,
        'over4500':0,}

    # Analyzing rests - Postconditioning
    postcond_rest_time_counter_k = {
        'under5':0,
        '5-10':0,
        '10-15':0,
        '15-20':0,
        '20-25':0,
        '25-30':0,
        '30-35':0,
        '35-40':0,
        '40-45':0,
        '45-50':0,
        '50-55':0,
        '55-60':0,
        '60-90':0,
        '90-450':0,
        '450-1800':0,
        '1800-4500':0,
        'over4500':0,}

    for sentence in postcond_k[0]:
        split_sentence = sentence.split()
        for word_idx in range(len(split_sentence)):
            if split_sentence[word_idx] == action:
                preexp_rest_time_counter_k[split_sentence[word_idx+1]] += 1

    for sentence in preexposure_k[0]:
        split_sentence = sentence.split()
        for word_idx in range(len(split_sentence)):
            if split_sentence[word_idx] == action:
                postcond_rest_time_counter_k[split_sentence[word_idx+1]] += 1

    preexp_rest_times = [preexp_rest_time_counter_k[x] for x in preexp_rest_time_counter_k.keys()]
    postcond_rest_times = [postcond_rest_time_counter_k[x] for x in postcond_rest_time_counter_k.keys()]

    X = [x for x in preexp_rest_time_counter.keys()]
    X_axis = np.arange(len(X))

    plt.figure(figsize=(30, 15))
    plt.rcParams.update({'font.size': 22})
    plt.bar(X_axis - 0.1, preexp_rest_times, 0.2, label = 'Preexposure')
    plt.bar(X_axis + 0.1, postcond_rest_times, 0.2, label = 'Postcond')

    plt.xticks(X_axis, X)
    plt.xlabel("Time Buckets of "+action+" in seconds")
    plt.ylabel("Frequency")
    plt.title("Action Frequency of "+action+" in Control Mouse")
    plt.legend()
    plt.show()

In [ ]:
def plotting_action_frequencies_c(action):
    # Analyzing rests - Preexposure
    preexp_rest_time_counter_c = {
        'under5':0,
        '5-10':0,
        '10-15':0,
        '15-20':0,
        '20-25':0,
        '25-30':0,
        '30-35':0,
        '35-40':0,
        '40-45':0,
        '45-50':0,
        '50-55':0,
        '55-60':0,
        '60-90':0,
        '90-450':0,
        '450-1800':0,
        '1800-4500':0,
        'over4500':0,}

    # Analyzing rests - Postconditioning
    postcond_rest_time_counter_c = {
        'under5':0,
        '5-10':0,
        '10-15':0,
        '15-20':0,
        '20-25':0,
        '25-30':0,
        '30-35':0,
        '35-40':0,
        '40-45':0,
        '45-50':0,
        '50-55':0,
        '55-60':0,
        '60-90':0,
        '90-450':0,
        '450-1800':0,
        '1800-4500':0,
        'over4500':0,}

    for sentence in postcond_c[0]:
        split_sentence = sentence.split()
        for word_idx in range(len(split_sentence)):
            if split_sentence[word_idx] == action:
                preexp_rest_time_counter_c[split_sentence[word_idx+1]] += 1

    for sentence in preexposure_c[0]:
        split_sentence = sentence.split()
        for word_idx in range(len(split_sentence)):
            if split_sentence[word_idx] == action:
                postcond_rest_time_counter_c[split_sentence[word_idx+1]] += 1

    preexp_rest_times = [preexp_rest_time_counter_c[x] for x in preexp_rest_time_counter_c.keys()]
    postcond_rest_times = [postcond_rest_time_counter_c[x] for x in postcond_rest_time_counter_c.keys()]

    X = [x for x in preexp_rest_time_counter.keys()]
    X_axis = np.arange(len(X))

    plt.figure(figsize=(30, 15))
    plt.rcParams.update({'font.size': 22})
    plt.bar(X_axis - 0.1, preexp_rest_times, 0.2, label = 'Preexposure')
    plt.bar(X_axis + 0.1, postcond_rest_times, 0.2, label = 'Postcond')

    plt.xticks(X_axis, X)
    plt.xlabel("Time Buckets of "+action+" in seconds")
    plt.ylabel("Frequency")
    plt.title("Action Frequency of "+action+" in Experiment Mouse")
    plt.legend()
    plt.show()

In [ ]:
def plotting_action_frequencies_b(action):
    # Analyzing rests - Preexposure
    preexp_rest_time_counter_b = {
        'under5':0,
        '5-10':0,
        '10-15':0,
        '15-20':0,
        '20-25':0,
        '25-30':0,
        '30-35':0,
        '35-40':0,
        '40-45':0,
        '45-50':0,
        '50-55':0,
        '55-60':0,
        '60-90':0,
        '90-450':0,
        '450-1800':0,
        '1800-4500':0,
        'over4500':0,}

    # Analyzing rests - Postconditioning
    postcond_rest_time_counter_b = {
        'under5':0,
        '5-10':0,
        '10-15':0,
        '15-20':0,
        '20-25':0,
        '25-30':0,
        '30-35':0,
        '35-40':0,
        '40-45':0,
        '45-50':0,
        '50-55':0,
        '55-60':0,
        '60-90':0,
        '90-450':0,
        '450-1800':0,
        '1800-4500':0,
        'over4500':0,}

    for sentence in postcond_b[0]:
        split_sentence = sentence.split()
        for word_idx in range(len(split_sentence)):
            if split_sentence[word_idx] == action:
                preexp_rest_time_counter_b[split_sentence[word_idx+1]] += 1

    for sentence in preexposure_b[0]:
        split_sentence = sentence.split()
        for word_idx in range(len(split_sentence)):
            if split_sentence[word_idx] == action:
                postcond_rest_time_counter_b[split_sentence[word_idx+1]] += 1

    preexp_rest_times = [preexp_rest_time_counter_b[x] for x in preexp_rest_time_counter_b.keys()]
    postcond_rest_times = [postcond_rest_time_counter_b[x] for x in postcond_rest_time_counter_b.keys()]

    X = [x for x in preexp_rest_time_counter.keys()]
    X_axis = np.arange(len(X))

    plt.figure(figsize=(30, 15))
    plt.rcParams.update({'font.size': 22})
    plt.bar(X_axis - 0.1, preexp_rest_times, 0.2, label = 'Preexposure')
    plt.bar(X_axis + 0.1, postcond_rest_times, 0.2, label = 'Postcond')

    plt.xticks(X_axis, X)
    plt.xlabel("Time Buckets of "+action+" in seconds")
    plt.ylabel("Frequency")
    plt.title("Action Frequency of "+action+" in Experiment Mouse")
    plt.legend()
    plt.show()

In [ ]:
def plotting_action_frequencies(action):
    # Analyzing rests - Preexposure
    preexp_rest_time_counter = {
        'under5':0,
        '5-10':0,
        '10-15':0,
        '15-20':0,
        '20-25':0,
        '25-30':0,
        '30-35':0,
        '35-40':0,
        '40-45':0,
        '45-50':0,
        '50-55':0,
        '55-60':0,
        '60-90':0,
        '90-450':0,
        '450-1800':0,
        '1800-4500':0,
        'over4500':0,}

    # Analyzing rests - Postconditioning
    postcond_rest_time_counter = {
        'under5':0,
        '5-10':0,
        '10-15':0,
        '15-20':0,
        '20-25':0,
        '25-30':0,
        '30-35':0,
        '35-40':0,
        '40-45':0,
        '45-50':0,
        '50-55':0,
        '55-60':0,
        '60-90':0,
        '90-450':0,
        '450-1800':0,
        '1800-4500':0,
        'over4500':0,}

    for sentence in postcond[0]:
        split_sentence = sentence.split()
        for word_idx in range(len(split_sentence)):
            if split_sentence[word_idx] == action:
                preexp_rest_time_counter[split_sentence[word_idx+1]] += 1

    for sentence in preexposure[0]:
        split_sentence = sentence.split()
        for word_idx in range(len(split_sentence)):
            if split_sentence[word_idx] == action:
                postcond_rest_time_counter[split_sentence[word_idx+1]] += 1

    preexp_rest_times = [preexp_rest_time_counter[x] for x in preexp_rest_time_counter.keys()]
    postcond_rest_times = [postcond_rest_time_counter[x] for x in postcond_rest_time_counter.keys()]

    X = [x for x in preexp_rest_time_counter.keys()]
    X_axis = np.arange(len(X))

    plt.figure(figsize=(30, 15))
    plt.rcParams.update({'font.size': 22})
    plt.bar(X_axis - 0.1, preexp_rest_times, 0.2, label = 'Preexposure')
    plt.bar(X_axis + 0.1, postcond_rest_times, 0.2, label = 'Postcond')

    plt.xticks(X_axis, X)
    plt.xlabel("Time Buckets of "+action+" in seconds")
    plt.ylabel("Frequency")
    plt.title("Action Frequency of "+action+" in Experiment Mouse")
    plt.legend()
    plt.show()

In [ ]:
plotting_action_frequencies_d("groom")

In [ ]:
plotting_action_frequencies_c("rest")

In [ ]:
plotting_action_frequencies_b("rest")

In [ ]:
plotting_action_frequencies("rest")

In [ ]:
plotting_action_frequencies_d("rear")

In [ ]:
plotting_action_frequencies_c("rear")

In [ ]:
plotting_action_frequencies_b("rear")

In [ ]:
plotting_action_frequencies("rear")

In [ ]:
plotting_action_frequencies_c("groom")

In [ ]:
plotting_action_frequencies("groom")

In [ ]:
plotting_action_frequencies("groom")

In [ ]:
plotting_action_frequencies_d("nibble")

In [ ]:
plotting_action_frequencies_c("nibble")

In [ ]:
plotting_action_frequencies("nibble")

In [ ]:
plotting_action_frequencies_d("hang")

In [ ]:
# Initializing a BERT bert-base-uncased style configuration
configuration = BertConfig()#vocab_size = 20, hidden_size = 24, num_hidden_layers = 6,  num_attention_heads = 12, intermediate_size = 1024, hidden_act = 'gelu', hidden_dropout_prob = 0.1, attention_probs_dropout_prob = 0.1,  max_position_embeddings = 45, type_vocab_size = 2, initializer_range = 0.02, layer_norm_eps = 1e-12, pad_token_id = 0, position_embedding_type = 'absolute',  use_cache = True, classifier_dropout = None )

In [ ]:
# Initializing a model (with random weights) from the bert-base-uncased style configuration
model = BertModel(configuration)

# Accessing the model configuration
configuration = model.config

In [ ]:
# Define Vocabulary:
# > '0-5'
# > '5-15'
# > '15-45'
# > '45-90'
# > '90-450'
# > '450-1800'
# > '1800-4500'
# > 'over4500'
# > 'drink'
# > 'eat'
# > 'groom'
# > 'hang'
# > 'sniff'
# > 'rear'
# > 'rest'
# > 'walk'
# > 'eathand'
# > [CLS]
# > [MASK]
# > [SEP]

# TOTAL SIZE = 20

In [7]:
!cat nih_vocab.txt

[PAD]
[UNK]
[CLS]
[SEP]
[MASK]
under5
5-10
10-15
15-20
20-25
25-30
30-35
35-40
40-45
45-50
50-55
55-60
60-90
90-450
450-1800
1800-4500
over4500
drink
eat
groom
hang
sniff
rear
rest
walk
nibble

In [4]:
from tqdm import tqdm
from transformers import BertTokenizerFast

# repositor id for saving the tokenizer
tokenizer_id="bert-base-uncased-2022-habana"

# create a python generator to dynamically load the data
def batch_iterator(batch_size=100):
    for i in tqdm(range(0, len(raw_text), batch_size)):
        print("Batch:", raw_text[i : i + batch_size])
        yield raw_datasets[i : i + batch_size]

# create a tokenizer from existing one to re-use special tokens
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
from transformers import BertTokenizer

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer("nih_vocab.txt", do_basic_tokenize=False)
# tokenizer.add_tokens(["0-5", "10-15"])

example_text = 'walk under5 rest 5-10'
bert_input = tokenizer(example_text,padding='max_length', max_length = 512, 
                       truncation=True, return_tensors="pt")
print(bert_input)

{'input_ids': tensor([[ 2, 29,  5, 28,  6,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  

In [5]:
# tokenizer = tokenizer.train_new_from_iterator(text_iterator=batch_iterator(), vocab_size = 20)
# tokenizer.save_pretrained("tokenizer")

In [6]:
raw_text = np.load("all_30s_sequences.npy")

In [7]:
def print_rand_sentence():
    '''
    Displays the tokens and respective IDs of a random text sample
    '''
    index = random.randint(0, len(raw_text)-1)
    table = np.array([tokenizer.tokenize(raw_text[index]), 
                    tokenizer.convert_tokens_to_ids(tokenizer.tokenize(raw_text[index]))]).T
    print(tabulate(table,
                 headers = ['Tokens', 'Token IDs'],
                 tablefmt = 'fancy_grid'))

print_rand_sentence()

╒══════════╤═════════════╕
│ Tokens   │   Token IDs │
╞══════════╪═════════════╡
│ rest     │          28 │
├──────────┼─────────────┤
│ 5-10     │           6 │
├──────────┼─────────────┤
│ sniff    │          26 │
├──────────┼─────────────┤
│ 15-20    │           8 │
├──────────┼─────────────┤
│ walk     │          29 │
├──────────┼─────────────┤
│ 5-10     │           6 │
├──────────┼─────────────┤
│ sniff    │          26 │
├──────────┼─────────────┤
│ 90-450   │          18 │
├──────────┼─────────────┤
│ walk     │          29 │
├──────────┼─────────────┤
│ 5-10     │           6 │
├──────────┼─────────────┤
│ sniff    │          26 │
├──────────┼─────────────┤
│ 90-450   │          18 │
├──────────┼─────────────┤
│ walk     │          29 │
├──────────┼─────────────┤
│ 10-15    │           7 │
├──────────┼─────────────┤
│ sniff    │          26 │
├──────────┼─────────────┤
│ 90-450   │          18 │
├──────────┼─────────────┤
│ walk     │          29 │
├──────────┼─────────────┤
│

In [8]:
all_sequences = raw_text

In [ ]:
token_id = []
attention_masks = []

def preprocessing(input_text, tokenizer):
    return tokenizer.encode_plus(input_text, add_special_tokens = True, max_length = 512,
                        pad_to_max_length = True, return_attention_mask = True, return_tensors = 'pt')

for sequence in all_sequences:
    encoding_dict = preprocessing(sequence, tokenizer)
    token_id.append(encoding_dict['input_ids']) 
    attention_masks.append(encoding_dict['attention_mask'])


token_id = torch.cat(token_id, dim = 0)
attention_masks = torch.cat(attention_masks, dim = 0)
labels = torch.tensor(labels)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/gpfs/data/tserre/anagara8/pytorch/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2304: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
len(token_id)

In [ ]:
val_ratio = 0.2
# Recommended batch size: 16, 32. See: https://arxiv.org/pdf/1810.04805.pdf
batch_size = 16
# Indices of the train and validation splits stratified by labels
train_idx, val_idx = train_test_split(
    np.arange(len(labels)),
    test_size = val_ratio,
    shuffle = True,
    stratify = labels)



# Train and validation sets
train_set = TensorDataset(token_id[train_idx], 
                          attention_masks[train_idx], 
                          labels[train_idx])

val_set = TensorDataset(token_id[val_idx], 
                        attention_masks[val_idx], 
                        labels[val_idx])

# Prepare DataLoader
train_dataloader = DataLoader(
            train_set,
            sampler = RandomSampler(train_set),
            batch_size = batch_size)

validation_dataloader = DataLoader(
            val_set,
            sampler = SequentialSampler(val_set),
            batch_size = batch_size)

In [ ]:
def b_tp(preds, labels):
    '''Returns True Positives (TP): count of correct predictions of actual class 1'''
    return sum([preds == labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_fp(preds, labels):
    '''Returns False Positives (FP): count of wrong predictions of actual class 1'''
    return sum([preds != labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_tn(preds, labels):
    '''Returns True Negatives (TN): count of correct predictions of actual class 0'''
    return sum([preds == labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_fn(preds, labels):
    '''Returns False Negatives (FN): count of wrong predictions of actual class 0'''
    return sum([preds != labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_metrics(preds, labels):
    '''
    Returns the following metrics:
    - accuracy    = (TP + TN) / N
    - precision   = TP / (TP + FP)
    - recall      = TP / (TP + FN)
    - specificity = TN / (TN + FP)
    '''
    preds = np.argmax(preds, axis = 1).flatten()
    labels = labels.flatten()
    tp = b_tp(preds, labels)
    tn = b_tn(preds, labels)
    fp = b_fp(preds, labels)
    fn = b_fn(preds, labels)
    b_accuracy = (tp + tn) / len(labels)
    b_precision = tp / (tp + fp) if (tp + fp) > 0 else 'nan'
    b_recall = tp / (tp + fn) if (tp + fn) > 0 else 'nan'
    b_specificity = tn / (tn + fp) if (tn + fp) > 0 else 'nan'
    return b_accuracy, b_precision, b_recall, b_specificity

In [ ]:
# Load the BertForSequenceClassificationjkk model
model = BertForSequenceClassification()#.from_pretrained(
#     'bert-base-uncased',
#     num_labels = 2,
#     output_attentions = False,
#     output_hidden_states = False,
# )

# Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5. See: https://arxiv.org/pdf/1810.04805.pdf
optimizer = torch.optim.AdamW(model.parameters(), 
                              lr = 5e-5,
                              eps = 1e-08
                              )

# Run on GPU
model.cuda()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Recommende number of epochs: 2, 3, 4. See: https://arxiv.org/pdf/1810.04805.pdf
epochs = 20

val_acc_overall = []
train_loss_overall = []

for _ in trange(epochs, desc = 'Epoch'):
    
    # ========== Training ==========
    
    # Set model to training mode
    model.train()
    
    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        optimizer.zero_grad()
        # Forward pass
        train_output = model(b_input_ids, 
                             token_type_ids = None, 
                             attention_mask = b_input_mask, 
                             labels = b_labels)
        # Backward pass
        train_output.loss.backward()
        optimizer.step()
        # Update tracking variables
        tr_loss += train_output.loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    # ========== Validation ==========

    # Set model to evaluation mode
    model.eval()

    # Tracking variables 
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_specificity = []

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
          # Forward pass
          eval_output = model(b_input_ids, 
                              token_type_ids = None, 
                              attention_mask = b_input_mask)
        logits = eval_output.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Calculate validation metrics
        b_accuracy, b_precision, b_recall, b_specificity = b_metrics(logits, label_ids)
        val_accuracy.append(b_accuracy)
        # Update precision only when (tp + fp) !=0; ignore nan
        if b_precision != 'nan': val_precision.append(b_precision)
        # Update recall only when (tp + fn) !=0; ignore nan
        if b_recall != 'nan': val_recall.append(b_recall)
        # Update specificity only when (tn + fp) !=0; ignore nan
        if b_specificity != 'nan': val_specificity.append(b_specificity)

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    train_loss_overall.append(tr_loss)
    print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    val_acc_overall.append(val_accuracy)
    print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)) if len(val_precision)>0 else '\t - Validation Precision: NaN')
    print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)) if len(val_recall)>0 else '\t - Validation Recall: NaN')
    print('\t - Validation Specificity: {:.4f}\n'.format(sum(val_specificity)/len(val_specificity)) if len(val_specificity)>0 else '\t - Validation Specificity: NaN')


In [ ]:
all_preexposure_actions_17202338_a

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


# category_names = ['Strongly disagree', 'Disagree',
#                   'Neither agree nor disagree', 'Agree', 'Strongly agree']
BEH_LABELS = ['drink', 'eat', 'groom', 'hang', 'sniff', 'rear', 'rest', 'walk', 'nibble']
action_colours = ['tab:blue', 'tab:orange', 'limegreen', 'deepskyblue', 'tab:pink', 'tab:purple', 'tab:brown', 'red', 'black']
results = {
    'Question 1': [10, 15, 17, 32, 26],
}


def survey(results, category_names):
    """
    Parameters
    ----------
    results : dict
        A mapping from question labels to a list of answers per category.
        It is assumed all lists contain the same number of entries and that
        it matches the length of *category_names*.
    category_names : list of str
        The category labels.
    """
    labels = list(results.keys())
    data = np.array(list(results.values()))
    data_cum = data.cumsum(axis=1)
    category_colors = plt.colormaps['RdYlGn'](
        np.linspace(0.15, 0.85, data.shape[1]))

    fig, ax = plt.subplots(figsize=(9.2, 1))
    ax.invert_yaxis()
    ax.xaxis.set_visible(False)
    ax.set_xlim(0, np.sum(data, axis=1).max())

    for i, (colname, color) in enumerate(zip(BEH_LABELS, action_colours)):
        widths = data[:, i]
        starts = data_cum[:, i] - widths
        rects = ax.barh(labels, widths, left=starts, height=0.5,
                        label=colname, color=color)

#         r, g, b, _ = color
        text_color = color#'white' if r * g * b < 0.5 else 'darkgrey'
        ax.bar_label(rects, label_type='center', color=text_color)
    ax.legend(ncol=len(category_names), bbox_to_anchor=(0, 1),
              loc='lower left', fontsize='small')

    return fig, ax


survey(results, category_names)
plt.show()

In [ ]:
import pylab as pl
import numpy as np

a = np.array([[0,1, 3, 5, 7]])
pl.figure(figsize=(9, 1.5))
img = pl.imshow(a, cmap="Blues")
pl.gca().set_visible(False)
cax = pl.axes([0.1, 0.2, 0.8, 0.6])
pl.colorbar(orientation="horizontal", cax=cax)

In [ ]:
a.iloc[97720]

In [ ]:
BEH_LABELS

In [ ]:
action_colours

In [ ]:
c = b[0:100000]

In [ ]:
n = 100000
plt.figure(figsize=(30, 1))
barlist=plt.bar(range(1,n+1), [1*n])
for i in range(n):
    barlist[i].set_color(action_colours[c[i]])
plt.xticks([])
plt.yticks([])
plt.show()